In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from configs.paths import abfss_path, checkpoint_path

In [0]:
df = spark.read.format("parquet")\
    .load(abfss_path("bronze", "customers"))

In [0]:
df = df.drop("_rescued_data")


In [0]:
df = df.withColumn("domains", split(col("email"), "@")[1])

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort(desc("total_customers")).display()

In [0]:
df_gmail = df.filter(col("domains")=="gmail.com")
df_yahoo = df.filter(col("domains")=="yahoo.com")
df_hotmail = df.filter(col("domains")=="hotmail.com")

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
df = df.drop("first_name", "last_name")

In [0]:
df.write.format("delta").mode("overwrite").save(abfss_path("silver", "customers"))

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS databricks_cata.silver")
spark.sql(f"""
CREATE TABLE IF NOT EXISTS databricks_cata.silver.customers_silver
USING DELTA
LOCATION '{abfss_path("silver", "customers")}'
""")